In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from lib.DrugUtil import flatten, DrugUtil

import plotly.express as px
from lib.FhirDataUtil import FhirDataUtil
from lib.CCSDataUtil import CCSDataUtil

def printmd(string):
    display(string)

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

_fhir_conditions_file = "input/fhir_conditions_20221003.csv"
_fhir_medications_file = "input/fhir_medications_20221003.csv"
_fhir_medications_ref_file = "input/fhir_medications_references_20221003.csv"
_fhir_demographics_file = "input/fhir_demographics_20221113.csv"
_output_dir = 'output/'

_rxnorm_file = "input_static/RxTerms202203/RxTerms202203.txt"
_rxnorm_ingredients_file = "input_static/RxTerms202203/RxTermsIngredients202203.txt"
_rxclass_file = "input_static/_rxclass_2022-04-10.csv"

_ccs_demographics_file = "input/ccs_demographics.csv"
_ccs_medications_file = "input/ccs_medications.csv"
_ccs_conditions_file = "input/ccs_conditions.csv"

_pc_demographics_file = "input/pcornet/demographic_fhir.csv"
_pc_medications_file = "input/pcornet/med_adm_fhir.csv"


In [4]:
drugUtil = DrugUtil()
drugUtil.load(rxnorm_file=_rxnorm_file,
                        rxnorm_ingredients_file=_rxnorm_ingredients_file, 
                        rxclass_file=_rxclass_file)

fhirUtil = FhirDataUtil()
ccsUtil = CCSDataUtil(drugUtil)

fhirUtil.load_conditions(fhir_conditions_file=_fhir_conditions_file, remove_health_concerns=True)
fhirUtil.load_demographics(fhir_demographics_file=_fhir_demographics_file)

ccsUtil.load_conditions(ccs_conditions_file= _ccs_conditions_file)
ccsUtil.load_demographics(_ccs_demographics_file)


Reading rxclass file...
Reading rxnorm file...
Loaded FHIR conditions records: 37248


/Users/bubbles/src/python/openepicvsr/lib/CCSDataUtil.py:13: DtypeWarning: Columns (5,6,11,12,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  self.conditions = pd.read_csv(ccs_conditions_file, delimiter=',')


Loaded CCS conditions records: 94735
Loaded demographics file with entries:  98877


/Users/bubbles/src/python/openepicvsr/lib/CCSDataUtil.py:75: DtypeWarning: Columns (11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  self.demographics = pd.read_csv(ccs_demographics)


In [5]:
snomed = fhirUtil.conditions[fhirUtil.conditions['coding_system'] == 'SNOMED']
print(f'Number of snomeds: {len(snomed)}')
icd10 = fhirUtil.conditions[fhirUtil.conditions['coding_system'] == 'ICD10']
print(f'Number of ICD10s: {len(icd10)}')


Number of snomeds: 19850
Number of ICD10s: 17398


# Conditions CCS-FHIR Intersection

In [4]:
len(fhirUtil.conditions_per_row)

12863

In [31]:
intersection = fhirUtil.conditions_per_row.merge(ccsUtil.conditions, on='user_id', how='inner')
intersection

,user_id,resourcetype,provider,uploaded_date,fhir_version,condition_text,category,resource_id,start,end,recordeddate,verification_text,status,ICD10,SNOMED,evisit,dt,survey_version,med_cond,hbp,diabetes,blockages_in_your_coronary,heart_attack,chf,stroke,afib,sleep_apnea,copd,asthma,cancer,immunodeficiency,HIV,anemia,pregnant
0,20,Condition,openepic_shared_sutter_health,2022-08-11,R4,Anatomical narrow angle of both eyes,Problem List Item,e58WUtj8HmvrRlpwTph1zNC5LbIuENhXgR6ZxopEeZcg3,2016-06-09,2016-06-09,2016-06-09,Confirmed,Active,H40.033,419422001,1,2021-03-29 00:03:46,4,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0
1,20,Condition,openepic_shared_sutter_health,2022-08-11,R4,"Arthritis of knee, left",Problem List Item,e5P9R6tnwVflfH0ziieAaCNMlmIPuAJlVvfMTQLlr.lg3,2017-10-16,2017-10-16,2017-10-16,Confirmed,Active,M17.12,"1074851000119104, 371081002",1,2021-03-29 00:03:46,4,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0
2,20,Condition,openepic_shared_sutter_health,2022-08-11,R4,Atrial fibrillation,Problem List Item,eb.dv1a62DgQt-I2eCxLB-A3,2006-08-08,2006-08-08,2006-08-08,Confirmed,Resolved,I48.91,49436004,1,2021-03-29 00:03:46,4,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0
3,20,Condition,openepic_shared_sutter_health,2022-08-11,R4,CTS (carpal tunnel syndrome),Problem List Item,eR4Gedb3vkbRtdRUFEW1oEPk8PhmuIFEsWHVQa0IJRFQ3,2012-11-07,2012-11-07,2012-11-07,Confirmed,Resolved,G56.00,57406009,1,2021-03-29 00:03:46,4,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0
4,20,Condition,openepic_shared_sutter_health,2022-08-11,R4,Carpal tunnel syndrome,Problem List Item,eHhUwe9zN8zVYwF.WqPexzaAVhMDuz74i5Zr1eIpztII3,2012-12-28,2012-12-28,2012-12-28,Confirmed,Active,G56.00,57406009,1,2021-03-29 00:03:46,4,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12279,561408,Condition,openepic_shared_presbyterian_healthcare_services,2022-08-11,R4,Mood disorder (CMS-HCC),Problem List Item,eoihHxX66LV-vXk.86zqhJTSE5gXkhruYKW2ZyBLkOAk3,2021-02-20,2021-02-20,2021-02-20,Confirmed,Inactive,F39,46206005,1,2022-03-17 20:19:49,4,NaN,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
12280,561408,Condition,openepic_shared_presbyterian_healthcare_services,2022-08-11,R4,Routine infant or child health check,Problem List Item,eqHDA43Vt9NVlNKD5UEKmLg3,2011-12-07,2011-12-07,2011-12-07,Confirmed,Active,Z00.129,NaN,1,2022-03-17 20:19:49,4,NaN,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
12281,561408,Condition,openepic_shared_presbyterian_healthcare_services,2022-08-11,R4,"Stimulant use disorder, severe, in early remission (CMS-HCC)",Problem List Item,eLV1F2Fx63cn0VEXspS6T3z8ceuMSFEctIUuBzRUhT9s3,2020-10-17,2020-10-17,2020-10-17,Confirmed,Active,F15.21,NaN,1,2022-03-17 20:19:49,4,NaN,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
12282,561408,Condition,openepic_shared_presbyterian_healthcare_services,2022-08-11,R4,"Tobacco use disorder, severe",Problem List Item,eeEHlnlQiRor4sgqx1GSP8pkmvILfWldZ8QMw8ZWtUlg3,2020-10-17,2020-10-17,2020-10-17,Confirmed,Active,F17.200,NaN,1,2022-03-17 20:19:49,4,NaN,1.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [32]:
# https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search?terms=E11
# to_numeric_fields = ['diabetes', 'hbp', 'blockages_in_your_coronary', 'heart_attack', 'chf', 'stroke', 'afib', 'sleep_apnea', 'copd', 'asthma', 'cancer', 'immunodeficiency', 'HIV', 'anemia', 'pregnant']

result = fhirUtil.searchFHIRandCCS(search_icd10_codes=['M17'], # I20.0, I21, I22, I24, I25.2, I25.5
                 ccs_field='chf', 
                 ccs_yes_values=[1], 
                 ccs_no_values=[2], # np.nan
                 condition_title=None)

result['both_negative'].head(100)
# len(result['both'].merge(result['ccs_only'], on="user_id", how="outer")['user_id'].drop_duplicates())


TypeError: searchFHIRandCCS() missing 1 required positional argument: 'ccsUtil'

In [13]:
comparison_config = [
    {
        'title': 'Diabetes', 
        'codes': ['E10','E11'], 
        'ccs_field': 'diabetes', 
        'ccs_yes': [1], 
        'ccs_no': [2]
    },
#     {
#         'title': 'Diabetes General', 
#         'codes': ['E08','E09','E10','E11','E13'], 
#         'ccs_field': 'diabetes', 
#         'ccs_yes': [1], 
#         'ccs_no': [2]
#     },
    {
        'title': 'Hypertension', 
        'codes': ['I10'], 
        'ccs_field': 'hbp', 
        'ccs_yes': [1], 
        'ccs_no': [2]
    },
#     {
#         'title': 'Hypertension General', 
#         'codes': ['I10', 'I11'], 
#         'ccs_field': 'hbp', 
#         'ccs_yes': [1], 
#         'ccs_no': [2]
#     },
    {
        'title': 'Myocardial Infarction',
        'codes': ['I21','I22','I24'],
        'ccs_field': 'heart_attack',
        'ccs_yes': [1], 'ccs_no': [2]
    },
#     {
#         'title': 'Myocardial Infarction General',
#         'codes': ['I20.0', 'I21', 'I22', 'I24', 'I25.2', 'I25.5'],
#         'ccs_field': 'heart_attack',
#         'ccs_yes': [1], 'ccs_no': [2]
#     },
    {
        'title': 'Coronary Artery Disease',
        'codes': ['I20','I21','I22','I23','I24','I25'],
        'ccs_field': 'blockages_in_your_coronary',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'Heart Failure',
        'codes': ['I50'],
        'ccs_field': 'chf',
        'ccs_yes': [1], 'ccs_no': [2]
    },
#     {
#         'title': 'Heart Failure General',
#         'codes': ['I11.0','I13.0','I50','I97.13'],
#         'ccs_field': 'chf',
#         'ccs_yes': [1], 'ccs_no': [2]
#     },
    {
        'title': 'Stroke',
        'codes': ['I63'],
        'ccs_field': 'stroke',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'AFib',
        'codes': ['I48'],
        'ccs_field': 'afib',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'Sleep Apnea',
        'codes': ['G47.3'],
        'ccs_field': 'sleep_apnea',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'COPD',
        'codes': ['J44'],
        'ccs_field': 'copd',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'Asthma',
        'codes': ['J45'],
        'ccs_field': 'asthma',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'Cancer',
        'codes': ['C'],
        'ccs_field': 'cancer',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'Immunodeficiency',
        'codes': ['C'],
        'ccs_field': 'immunodeficiency',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'HIV',
        'codes': ['Z21', 'B20', 'B97.35', 'O98.7'],
        'ccs_field': 'HIV',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'Anemia',
        'codes': ['D50','D51','D52','D53.0','D53.2','D53.9','D64.9'],
        'ccs_field': 'anemia',
        'ccs_yes': [1], 'ccs_no': [2]
    },
    {
        'title': 'Anemia General',
        'codes': ['D46.0','D46.1','D46.4','D46.2','D50','D51','D52','D53.0','D53.2','D53.9','D58.9','D59','D61','D55','D63','D64.9', 'D64.81', 'D64.1', 'D64.2'],
        'ccs_field': 'anemia',
        'ccs_yes': [1], 'ccs_no': [2]
    }
]
# ['diabetes', 'hbp', 'blockages_in_your_coronary', 'heart_attack', 'chf', 'stroke', 'afib', 'sleep_apnea', 'copd', 'asthma', 'cancer', 'immunodeficiency', 'HIV', 'anemia', 'pregnant']



In [14]:
def compare(self, 
                     ccs_cond, 
                     fhir_cond, 
                     pc_cond, 
                     search_icd10_codes, 
                     ccs_field, 
                     ccs_yes_values=[1], 
                     ccs_no_values=[2], 
                     condition_title=None, 
                     verbose=True):
    print('test')
    
compare(ccs_cond, fhir_cond
    

SyntaxError: unexpected EOF while parsing (892460070.py, line 14)

In [15]:
def getFhirCCSComparisonTable(self, ccsUtil, comparison_config):
    result = []
      for config in comparison_config:
        print('Processing ', config['title'])
        comp = self.searchFHIRandCCS(
                                   ccsUtil=ccsUtil,
                                   search_icd10_codes=config['codes'], 
                                   ccs_field=config['ccs_field'], 
                                   ccs_yes_values=config['ccs_yes'], 
                                   ccs_no_values=config['ccs_no'],
                                   condition_title=config['title'],
                                   verbose=False)

         resultRow = {
            'title': config['title'],
            'Both Pos': comp['both_len'],
            'CF %': round((comp['both_len']*100)/comp['total_yes']),
            'CCS Pos': comp['ccs_only_len'],
            'C %': round((comp['ccs_only_len']*100)/comp['total_yes']),
            'FHIR Pos': comp['fhir_only_len'],
            'F %': round((comp['fhir_only_len']*100)/comp['total_yes']),
            'Both Negative': comp['both_negative_len'],
            'BN %': round((comp['both_negative_len']*100)/(comp['total_no']))
         }
    result.append(resultRow)
    result_df = pd.DataFrame(result)
    return result_df

result = fhirUtil.getFhirCCSComparisonTable(ccsUtil=ccsUtil, comparison_config=comparison_config)

IndentationError: unexpected indent (3792365161.py, line 3)

In [35]:
grey_columns = ['CF %', 'F %', 'C %', 'BN %']
red_columns = ['Both Negative']
def highlight_cols(x):
    df = x.copy()
    df.loc[:, :] = 'background-color: none'
    df[grey_columns] = 'color: grey'
    df[red_columns] = 'color: red'
    return df 
display(result.style.apply(highlight_cols, axis = None))

,title,Both Pos,CF %,CCS Pos,C %,FHIR Pos,F %,Both Negative,BN %
0,Diabetes,52,54,28,29,17,18,798,100
1,Diabetes General,52,54,28,29,17,18,798,100
2,Hypertension,225,60,110,30,37,10,524,100
3,Hypertension General,227,61,108,29,37,10,524,100
4,Myocardial Infarction,8,29,13,46,7,25,869,100
5,Myocardial Infarction General,11,37,10,33,9,30,868,100
6,Coronary Artery Disease,43,52,25,30,15,18,810,99
7,Heart Failure,4,18,13,59,5,23,876,100
8,Heart Failure General,5,23,12,55,5,23,876,100
9,Stroke,5,18,20,71,3,11,866,100


# PcorNet

In [6]:
pc_cond = pd.read_csv('input/pcornet/diagnosis_fhir.csv')
pc_demo = pd.read_csv('input/pcornet/demographic_fhir.csv')

/var/folders/_n/j7wb5zn173d8t_tnxz9rb5p00000gn/T/ipykernel_10275/4266438744.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  pc_cond = pd.read_csv('input/pcornet/diagnosis_fhir.csv')


In [9]:
fhir_demo = fhirUtil.demographics.drop_duplicates(subset="user_id")
fhir_cond = fhirUtil.conditions[fhirUtil.conditions['coding_system'] == 'ICD10']

In [12]:
ccs_cond = ccsUtil.conditions

In [10]:
m_demo = pc_demo.drop_duplicates(subset="user_id").merge(fhir_demo.drop_duplicates(subset="user_id"), on="user_id", indicator=True, how="outer")
common_users = m_demo[m_demo['_merge'] == 'both']
m_demo['_merge'].value_counts()


right_only    844
both          618
left_only      11
Name: _merge, dtype: int64

In [69]:
print('Common total fhir users:', len(fhir_demo[fhir_demo['user_id'].isin(common_users['user_id'])].drop_duplicates('user_id')))
print('Common total pcornet users:', len(pc_demo[pc_demo['user_id'].isin(common_users['user_id'])].drop_duplicates('user_id')))
fhir_cond_pc = fhir_cond[fhir_cond['user_id'].isin(common_users['user_id'])]
pc_cond_fhir = pc_cond[pc_cond['user_id'].isin(common_users['user_id'])]
print('Common fhir users with conditions: ', len(fhir_cond_pc.drop_duplicates('user_id')))
print('Common Pcornet users with conditions:', len(pc_cond_fhir.drop_duplicates('user_id')))

Common total fhir users: 618
Common total pcornet users: 618
Common fhir users with conditions:  600
Common Pcornet users with conditions: 611


In [102]:
def compare(left, right, codes, left_field, right_field, verbose=True):
#         {
#         'title': 'Diabetes', 
#         'codes': ['E10','E11'], 
#         'ccs_field': 'diabetes', 
#         'ccs_yes': [1], 
#         'ccs_no': [2]
#     },
    
    left_filter = left[left_field].str.startswith(tuple(codes))
    left_s = left[left_filter].drop_duplicates(subset='user_id')
    left_no_s = common_users[~common_users["user_id"].isin(left_s['user_id'])].drop_duplicates(subset='user_id')
    
    right_filter = right[right_field].str.startswith(tuple(codes))
    right_s = right[right_filter].drop_duplicates(subset="user_id")
    right_no_s = common_users[~common_users["user_id"].isin(right_s['user_id'])].drop_duplicates(subset='user_id')
    
    m = left_s.merge(right_s, on="user_id", how="outer", indicator=True)
    
    left_no_s.drop('_merge', axis=1, inplace=True)
    right_no_s.drop('_merge', axis=1, inplace=True)
    m_no = left_no_s.merge(right_no_s, on="user_id", how="outer", indicator=True)
    
    both_positive = m[m['_merge']=='both']
    left_only = m[m['_merge'] == 'left_only']
    right_only = m[m['_merge'] == 'right_only']
    
    both_negative = m_no[m_no['_merge'] == 'both']
    
    r = {
        'both_len': len(both_positive),
        'left_only_len': len(left_only),
        'right_only_len': len(right_only),
        'both_negative_len': len(both_negative),
        'total_yes': len(both_positive)+len(left_only)+len(right_only),
        'total_no': len(left_only)+len(right_only)+len(both_negative)
    }
    return r
    
    
result = []
for config in comparison_config:
    print('Processing ', config['title'])
    comp = compare(
                               left=fhir_cond_pc,
                               right=pc_cond_fhir,
                               codes=config['codes'], 
                               left_field= 'coding_code',
                               right_field='dx',
                               verbose=False)

    resultRow = {
    'title': config['title'],
    'Both Pos': comp['both_len'],
    'CF %': round((comp['both_len']*100)/comp['total_yes']),
    'FHIR Pos': comp['left_only_len'],
    'C %': round((comp['left_only_len']*100)/comp['total_yes']),
    'PCORNET Pos': comp['right_only_len'],
    'F %': round((comp['right_only_len']*100)/comp['total_yes']),
    'Both Negative': comp['both_negative_len'],
    'BN %': round((comp['both_negative_len']*100)/(comp['total_no']))
    }
    result.append(resultRow)
result_df = pd.DataFrame(result)

grey_columns = ['CF %', 'F %', 'C %', 'BN %']
red_columns = ['FHIR Pos', 'PCORNET Pos']
green_columns = ['Both Pos', 'Both Negative']
def highlight_cols(x):
    df = x.copy()
    df.loc[:, :] = 'background-color: none'
    df[grey_columns] = 'color: grey'
    df[red_columns] = 'color: red'
    df[green_columns] = 'color: green'
    return df 
display(result_df.style.apply(highlight_cols, axis = None))

# result_df


Processing  Diabetes
Processing  Diabetes General
Processing  Hypertension
Processing  Hypertension General
Processing  Myocardial Infarction
Processing  Myocardial Infarction General
Processing  Coronary Artery Disease
Processing  Heart Failure
Processing  Heart Failure General
Processing  Stroke
Processing  AFib
Processing  Sleep Apnea
Processing  COPD
Processing  Asthma
Processing  Cancer
Processing  Immunodeficiency
Processing  HIV
Processing  Anemia
Processing  Anemia General


,title,Both Pos,CF %,FHIR Pos,C %,PCORNET Pos,F %,Both Negative,BN %
0,Diabetes,46,56,11,13,25,30,536,94
1,Diabetes General,47,57,10,12,25,30,536,94
2,Hypertension,161,65,33,13,52,21,372,81
3,Hypertension General,161,65,33,13,54,22,370,81
4,Myocardial Infarction,3,23,5,38,5,38,605,98
5,Myocardial Infarction General,6,27,5,23,11,50,596,97
6,Coronary Artery Disease,31,47,6,9,29,44,552,94
7,Heart Failure,9,39,1,4,13,57,595,98
8,Heart Failure General,9,39,1,4,13,57,595,98
9,Stroke,2,29,2,29,3,43,611,99


In [70]:
# Check dataset
fhir_cond_pc
pc_cond_fhir

,v1,user_id,diagnosisid,encounterid,enc_type,admit_date,dx,dx_type,dx_date,dx_source,pdx,datamartid,siteid,query_date,cdrn_facilityid
0,26,502639,{C21B53E3-3272-44D4-B1F8-78F25CC597AC},1038398261,OA,14jun2020,J31.0,10,14jun2020,FI,S,SUTTER,SH2PARTNER,14jun2022,NaN
1,26,502639,{59185F2E-E7D9-46E3-A311-FAC5D67022FE},1136833765,OA,23nov2021,M79.18,10,23nov2021,FI,S,SUTTER,SH2PARTNER,14jun2022,NaN
2,26,502639,{C9E93863-78D8-4CDC-90F5-D7C65E02D443},892747432,AV,19jan2018,Z23,10,19jan2018,FI,S,SUTTER,SH2PARTNER,14jun2022,NaN
3,26,502639,{A5730640-15ED-411F-BD29-6181EB5C662D},936074887,AV,14sep2018,R92.2,10,14sep2018,FI,S,SUTTER,SH2PARTNER,14jun2022,NaN
4,26,502639,{7F719426-21A7-463B-84F0-1A29012FF1C3},1072911350,OA,30dec2020,I49.9,10,30dec2020,FI,S,SUTTER,SH2PARTNER,14jun2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120339,48641,547213,842348168,UTAH10581382,AV,10aug2017,Z00.00,10,NaN,FI,P,C4UU,UT1PARTNER,13jun2022,NaN
120340,48641,547213,548738098,UTAH11794463,AV,18dec2017,J01.00,10,NaN,FI,P,C4UU,UT1PARTNER,13jun2022,NaN
120341,48641,547213,25928655,UTAH10581382,AV,10aug2017,Z12.4,10,NaN,FI,S,C4UU,UT1PARTNER,13jun2022,NaN
120342,48641,547213,873757065,UTAH21035279,AV,03mar2021,Z23,10,NaN,FI,P,C4UU,UT1PARTNER,13jun2022,NaN


In [93]:
pc_users = pc_cond_fhir[pc_cond_fhir['dx'].str.startswith(('E10', 'E11'))].drop_duplicates(subset="user_id")
fhir_users = fhir_cond_pc[fhir_cond_pc['coding_code'].str.startswith(('E10', 'E11'))].drop_duplicates(subset="user_id")

In [96]:
len(fhir_users[~fhir_users['user_id'].isin(pc_users['user_id'])])

11

In [87]:
len(common_users[~common_users['user_id'].isin((*pc_users, *fhir_users))])

536

In [103]:
len([1647, 16853, 102891, 117086, 200552, 302382, 324260, 324726, 324961, 331019, 331110, 331135, 331300, 331450, 331467, 331515, 331920, 333749, 344968, 357029, 383858, 386617, 387173, 387417, 388923, 389802, 395485, 395659, 395756, 397160, 399087]
)

31